# Analysis of Trump vs Harris Interviews and Podcasts using Key Word in Context

Overview of Key Word in Context:

KWIC (Key Word in Context) analysis is a method used in text analysis to explore how specific words or phrases are used within their surrounding context in a corpus.

Steps:

* The `make_kwic()` function returns:
    * a `list` where each item is one concordance line containing
         1. a `list` of tokens to the left of the keyword
         2. the keyword
         3. a `list` of tokens to the right of the keyword
         
    * so we have a `list-of-lists` structure
 

* We pass `make_kwic`:
    1. a search term (or `keyword`)
    2. a `list` of tokens
    3. the size of the context


* Which you can see is itself a list with 3 items:
    * index 0 - left context tokens (a list of length win - here 4 words)
    * index 1 - the keyword as a string
    * index 2 - right context tokens (a list of length win - here 4 words)
    
* The `list-of-lists` structure is hard to read so we can write a `print_kwic()` function  to display the KWIC lines more nicely

This notebook is organized to guide the analysis step-by-step, starting with data preparation and progressing through multiple comparative analyses. Each section focuses on addressing one of our key research questons

Here’s an overview of the structure:

* Setup

* Loading the Data

* Understanding the Data Structure and Preparing for Analysis

* Issue Focus

* Opponent Mentions

* Emotional Language

### Setup

In [1]:
import json
import random
from collections import Counter
import matplotlib.pyplot as plt
import emoji

from nltk.tokenize import TweetTokenizer


In [2]:
def tokenize(text, lowercase=True, strip_chars=''):
    '''turn a string into a list of whitespace separated tokens - after observing lowercase flag and stripping specified characters
    
    Args:
        text        -- a string object containing the text to be tokenized
        lowercase   -- whether the string should be lowercased before tokenization (default: True)
        strip_chars -- a string containing a series of characters which should be stripped from text before tokenization (default: empty string)
        
    
    Returns:
        list of tokens
    '''
    if lowercase:
        text=text.lower()
        
    rdict = str.maketrans('','',strip_chars)
    text = text.translate(rdict)
        
    tokens=text.split()
    
    return tokens

In [3]:
def make_kwic(kw, text, win=4):
    '''A basic KWIC function for a text
    
    Args:
        kw   -- string match for keyword to match for each line
        text -- a list of tokens for the text
        win  -- window size for left and right context
        
    Return:
        list of lines of form [ [left context words], kw, [right context words]]
    '''
    hits = [(word, idx) for idx, word in enumerate(text) if word == kw]
    
    lines = []
    for hit in hits:
        hidx = hit[1]
        kw = hit[0]
        left = text[max(0, hidx-win):hidx]
        right = text[hidx+1:min(len(text), hidx+win+1)]
        
        left = [''] * (win - len(left)) + left if len(left) < win else left
        right = right + [''] * (win - len(right)) if len(right) < win else right
        
        lines.append([left, kw, right])
    
    return lines

In [4]:
def print_kwic(kwic, win=None):
    '''A basic print function for a KWIC object
    
    Args:
        kwic -- a list of KWIC lines of the form [ [left words], kw, [right words]]
        win  -- if None then use all words provided in context otherwise limit by win
        
    Prints KWIC lines with left context width/padding win*8 characters
    '''
    
    if not kwic:
        return
    
    if win is None:
        win = len(kwic[0][0])
        
    max_left = max([len(' '.join(l[0])) for l in kwic])
    
    for line in kwic:
        print("{: >{}}  {}  {}".format(' '.join(line[0][-win:]), 
                                      max_left, 
                                      line[1], 
                                      ' '.join(line[2][:win])
                                     )
             )            

In [5]:
def sort_kwic(kwic, order=None):
    ''' sort a kwic list using the passed positional arguments 
    
    Args:
        kwic   -- a list of lists [ [left tokens], kw, [right tokens]]
        order  -- a list of one or more positional arguments of form side-pos, e.g. L1, R3, L4 (default: None)
    
    Returns:
        kwic sorted for each positional argument in reverse, i.e. ['R1','L1'] sorts first by L1 and then R1
    '''
    if order is None:
        return kwic
   
    order = [order] if not type(order) is list else order
    order.reverse()
    
    for sort_term in order:
        if not re.match('[LR][1-4]', sort_term):
            pass
        
        pos1 = 0 if sort_term[0]=='L' else 2
        pos2 = int(sort_term[1])-1
        pos2 = 3-pos2 if sort_term[0]=='L' else pos2
        kwic.sort(key=lambda l : l[pos1][pos2])
    
    return kwic

### Loading the Data

In [6]:
master_list = json.load(open('../data/master_list.json'))

### Understanding the Data Structure and Preparing for Analysis

In [7]:
type(master_list)

list

In [8]:
len(master_list)

24

In [9]:
type(master_list[0])

dict

In [10]:
master_list[1]

{'title': 'Harris_CNN',
 'medium': 'harris_interviews',
 'transcript': [['Host',
   'Madam Vice President, Governor Walz, thank you so much for sitting down with me and bringing the bus. The bus tour is well underway here in Georgia. You have less time to make your case to voters than any candidate in modern American history. The voters are really eager to hear what your plans are. If you are elected, what would you do on day one in the White House?'],
  ['Harris',
   'Well, there are a number of things. I will tell you first and foremost, one of my highest priorities is to do what we can to support and strengthen the middle class. When I look at the aspirations, the goals, the ambitions of the American people, I think that people are ready for a new way forward in a way that generations of Americans have been fueled by hope and by optimism. I think, sadly, in the last decade, we have had in the former president someone who has really been pushing an agenda and an environment that is a

In [11]:
# Separate lists for each type of interview or podcast
harris_interviews_toks = []
harris_podcasts_toks = []
trump_interviews_toks = []
trump_podcasts_toks = []

# Filter and append tokens for Harris interviews
for item in master_list:
    if item['medium'] == 'harris_interviews':
        harris_interviews_toks.extend(item['tokens'])
    elif item['medium'] == 'harris_podcasts':
        harris_podcasts_toks.extend(item['tokens'])
    elif item['medium'] == 'trump_interviews':
        trump_interviews_toks.extend(item['tokens'])
    elif item['medium'] == 'trump_podcasts':
        trump_podcasts_toks.extend(item['tokens'])

**To address our key research questions, we have chosen several words to analyze using KWIC.**

Issue Focus: % of Issue-Focused Content vs Personal Anecdotes
* Words to Analyze: Border, Inflation
   
Opponent Mentions: Which candidate mentions the other more frequently?
* Words to Analyze: 'Trump,' 'Harris'

Emotional Language: In which outlet is emotional language utilized more?
* Words to Analyze: 'Very', 'Good', 'Bad'


### Issue Focus

'Border' mentions in Trump Interviews

In [12]:
border_trump_interviews_kwic = make_kwic('border', trump_interviews_toks)
border_trump_random_sample = random.sample(border_trump_interviews_kwic, min(30, len(border_trump_interviews_kwic)))
print_kwic(border_trump_random_sample)

              so she was a  border  czar shes done a
       even better but the  border  cant i mean we
      theyre very near the  border  and theyre going to
      people killed in the  border  look at the afghanistan
       did things with the  border  that nobody could have
       czar shes the worst  border  czar in the history
        to happen shes the  border  czar shes the worst
      that coming from the  border  are millions and millions
    got the endorsement of  border  patrol all over i
       theres never been a  border  czar like this shes
         i left office the  border  was the safest it
            do i close the  border  and i do two
simultaneously i close the  border  we dont want people
           and we have the  border  were the safest border
         going to make the  border  even better but the
            we never had a  border  that safe the famous
      we had the strongest  border  in the history of
      all over the country  border  patrol virtually

**Observations**

* Criticism of Harris ("she's the worst border czar", "never been a border czar like this")
* Personal Responsibility ("I left office the border was the safest", "We did things with the border that nobody could have")
* Unsafe Border Under Others ("We have the most unsafe border", "Millions and millions coming from the border")


'Border' mentions in Trump Podcasts

In [13]:
border_trump_podcasts_kwic = make_kwic('border', trump_podcasts_toks)
border_trump_podcasts_random_sample = random.sample(border_trump_podcasts_kwic, min(30, len(border_trump_podcasts_kwic)))
print_kwic(border_trump_podcasts_random_sample)

          she wasnt really the  border  czar but she was
               good job on the  border  we had the worst
           gonna make a strong  border  they say theyve been
             going to seal the  border  and i had it
               and i fixed the  border  and i couldnt even
             we had the safest  border  and the best border
           border and the best  border  in the history of
      with the most successful  border  in the history of
             of the border our  border  is 25 times worse
            is the most unsafe  border  anywhere in the world
different things including the  border  et cetera et cetera
    comes through the southern  border  and i was with
                 on the on the  border  its such a lie
        whats happening at our  border  joe when you have
          think because of the  border  our border is 25
              in charge of the  border  and we have her
        couldnt talk about the  border  because i fixed it
           i had 

**Observations**

* Criticism of Current Admin: ("The border was the worst ever", "Really bad the worst border in history")
* Trump's Succesess: ("I ran on the border and I fixed it", "I fixed the border and I couldn’t even", "Very strong at the border")

'Border' mentions in Harris Interviews

In [14]:
border_harris_interviews_kwic = make_kwic('border', harris_interviews_toks)
border_harris_interviews_random_sample = random.sample(border_harris_interviews_kwic, min(30, len(border_harris_interviews_kwic)))
print_kwic(border_harris_interviews_random_sample)

           would put 1500 more  border  agents at the border
         who actually served a  border  state as attorney general
        tell you something the  border  patrol endorsed the bill
   contributed to securing our  border  and because he believes
             why i believe the  border  patrol agents supported the
          people and cross our  border  with the intent of
          border agents at the  border  to give those folks
       human beings across our  border  my values have not
       points of entry between  border  entry points thats the
              just down at the  border  talking with border agents
          border agents at the  border  which is why i
                true that if a  border  security had actually been
not believe in decriminalizing  border  crossings and ive not
            have put 1500 more  border  agents at the border
       the border talking with  border  agents and they will
              do to secure our  border  that value has not
   

**Observations**

* Focus on Legislative Action: ("Border security bill that would", "Came up with a border security bill", "That border bill would have put")
* Support for law enforcement: ("Border patrol agents supported the", "Talking with border agents", "What the customs and border patrol agents need")
* Strong stance on illegal crossing: ("Not believe in decriminalizing border crossings", "People who cross our border illegally", "Harm and cross our border illegally")

‘Border’ mentions in Harris Podcasts

In [15]:
border_harris_podcasts_kwic = make_kwic('border', harris_podcasts_toks)
border_harris_podcasts_random_sample = random.sample(border_harris_podcasts_kwic, min(30, len(border_harris_podcasts_kwic)))
print_kwic(border_harris_podcasts_random_sample)

        about having a secure  border  and making sure that
         border agents at the  border  ive been down to
         border agents at the  border  it would have reduced
           have put 1500 more  border  agents at the border
      we gotta strengthen the  border  and we need to
         put more support for  border  patrol agents put more
cares about strengthening the  border  when he had an
      have been the strongest  border  security bill in years
           have put 1500 more  border  agents at the border
         border agents at the  border  to do what we
             try and get that  border  security bill passed and
          to the border those  border  agents are working around
             been down to the  border  those border agents are
             to put 1500 more  border  agents at the border
       more technology at the  border  but it includes also
               came up with a  border  security bill which was
        more resources at the  border  whic

**Observations**

* Focus on legislature: ("Strongest border security bill in years", "Border security bill which was", "Try and get that border security bill passed")
* Specific actions in bills: ("Put 1500 more border agents at the border", "More resources at the border", "More support for border patrol agents")

‘Inflation’ mentions in Trump Interviews

In [16]:
inflation_trump_interviews_kwic = make_kwic('inflation', trump_interviews_toks)
inflation_trump_interviews_random_sample = random.sample(inflation_trump_interviews_kwic, min(30, len(inflation_trump_interviews_kwic)))
print_kwic(inflation_trump_interviews_random_sample)

           check that too but  inflation  harris is absolutely destroying
           rates down i bring  inflation  way down so people
population within our country  inflation  is the worst its
            had four years no  inflation  i had four years
             down the cost of  inflation  this was all started
            two years with no  inflation  because he inherited from
      disaster in our country  inflation  is a country buster
  class virtually every class  inflation  is a disaster in
building because of inflation  inflation  is a disaster and
             done to you with  inflation  is a disaster its
            had four years no  inflation  i had four years
             we had the worst  inflation  in the history of
          the numbers its the  inflation  we had the worst
               a mess we have  inflation  we have the millions
              can with it the  inflation  is absolutely destroying our
         my opinion the worst  inflation  weve had they say
   

**Observations**

* Low inflation during Trump's presidency: ("Had four years no inflation", "Two years with no inflation")
* Criticism of Current levels: ("Inflation is a disaster", "Inflation is the worst its been"
, "Inflation is absolutely destroying our")


‘Inflation’ mentions in Trump Podcasts

In [17]:
inflation_trump_podcasts_kwic = make_kwic('inflation', trump_podcasts_toks)
inflation_trump_podcasts_random_sample = random.sample(inflation_trump_podcasts_kwic, min(30, len(inflation_trump_podcasts_kwic)))
print_kwic(inflation_trump_podcasts_random_sample)

       just dont understand where  inflation  comes from inflation comes
             too with energy with  inflation  hes been a disaster
        of people concerned about  inflation  and inflation is effectively
           problem well solve the  inflation  problem which means people
               in this company no  inflation  we would have been
      disaster with inflation the  inflation  it doesnt matter what
                  and wed have no  inflation  and we wouldnt have
    concerned about inflation and  inflation  is effectively a tax
         goods and services thats  inflation  so really we need
             to buy groceries the  inflation  has killed them food
               would also have no  inflation  was caused by oil
          look what happened with  inflation  because inflations just eating
        would have never happened  inflation  would have never happened
          have never happened and  inflation  would have never happened
        destroyed our country t

**Observations**

* Emphasis on the impact of Inflation: ("Inflation has killed them food", "Inflation is eating you alive", "Inflation is effectively a tax")
* Contrast with Trump's leadership: ("We'd have no inflation", "Inflation would have never happened", "We would have been")

‘Inflation’ mentions in Harris Interviews

In [18]:
inflation_harris_interviews_kwic = make_kwic('inflation', harris_interviews_toks)
inflation_harris_interviews_random_sample = random.sample(inflation_harris_interviews_kwic, min(30, len(inflation_harris_interviews_kwic)))
print_kwic(inflation_harris_interviews_random_sample)

      them weaker would ignite  inflation  and invite a recession
  on infrastructure before the  inflation  reduction act before the
         done that has brought  inflation  down to less than
have estimated will exasperate  inflation  and invite a recession
             know that we have  inflation  at under 3 percent
     accomplished thus far the  inflation  reduction act what we
             did that with the  inflation  reduction act we have


**Observations**
* Emphasis on achievements: ("Inflation at under 3 percent", "Brought inflation down to less than")
* Avoiding recession: ("Ignite inflation and invite a recession", "Estimated will exasperate inflation and invite a recession")

‘Inflation’ mentions in Harris Podcasts

In [19]:
inflation_harris_podcasts_kwic = make_kwic('inflation', harris_podcasts_toks)
inflation_harris_podcasts_random_sample = random.sample(inflation_harris_podcasts_kwic, min(30, len(inflation_harris_podcasts_kwic)))
print_kwic(inflation_harris_podcasts_random_sample)

 the economy would accelerate  inflation  and invite a recession
     been wages have outpaced  inflation  inflation is going down
    our economy would inflate  inflation  and would bring a
            before we did the  inflation  reduction act before we
wages have outpaced inflation  inflation  is going down to


**Observations**
* Framing of economic trends: ("Wages have outpaced inflation", "Inflation is going down")
* Preventing economic downturn: ("Inflate inflation and would bring a", "Accelerate inflation and invite a recession")

### Opponent Mentions

'Harris' mentions in Trump Interviews

In [20]:
harris_trump_interviews_kwic = make_kwic('harris', trump_interviews_toks)
harris_trump_interviews_random_sample = random.sample(harris_trump_interviews_kwic, min(30, len(harris_trump_interviews_kwic)))
print_kwic(harris_trump_interviews_random_sample)

      that too but inflation  harris  is absolutely destroying this
       states senator and so  harris  like i respect you
there because because kamala  harris  whos our borders are
        it again i suggested  harris  that — lets take
           people to come in  harris  but they have to
        a few more questions  harris  because of the distance
       lady right over there  harris  is a fantastic person


**Observations**

* Critical Commentary: ('destroying this')
* Positive Description ('fantastic person')
* Overall neutral tone

'Harris' mentions in Trump Podcasts

In [21]:
harris_trump_podcasts_kwic = make_kwic('k', trump_podcasts_toks)
harris_trump_podcasts_random_sample = random.sample(harris_trump_podcasts_kwic, min(30, len(harris_trump_podcasts_kwic)))
print_kwic(harris_trump_podcasts_random_sample)


         this can you imagine  kamala  doing this show if
       people would she rests  kamala  rests in fact shes
        think you have strong  kamala  voters whats to vote
        are some like comrade  kamala  its a little hard
        biden and kamala lyin  kamala  theyve done a bad
          should he should do  kamala  that would be beautiful
           ive seen the worst  kamala  goes on 60 minutes
            years youre not a  kamala  person youre a khabib
   happened with these people  kamala  and joe you see
              when they see a  kamala  or when they see
          laying on the floor  kamala  youd be saying call
 unconstitutional but i think  kamala  should have a test
        percent and biden and  kamala  didnt press it it
the biden administration with  kamala  is accusing me of
            maybe ill call it  kamala  migrant crime the only
           kamala youre not a  kamala  person i know you
         cetera that was okay  kamala  is a very low
           you

'Trump' mentions in Harris Interviews

In [22]:
trump_harris_interviews_kwic = make_kwic('trump', harris_interviews_toks)
trump_harris_interviews_random_sample = random.sample(trump_harris_interviews_kwic, min(30, len(trump_harris_interviews_kwic)))
print_kwic(trump_harris_interviews_random_sample)

            month for seniors donald  trump  said he was going
         rhetoric coming from donald  trump  that has been designed
                    that ad from the  trump  campaign is a little
           has happened since donald  trump  allowed roe v wade
russias unprovoked aggression donald  trump  if he were president
                   go back to donald  trump  who pulled out of
          guess what happened donald  trump  got word that this
               has said about donald  trump  being a threat to
                  come out of donald  trump  people are ready to
              just finish and donald  trump  learned about that bill
                   20 states we have  trump  abortion bans which some
           are exhausted with donald  trump  and his approach because
                   as i think donald  trump  would say he did
           brain injuries and donald  trump  dismissed them as headaches
               between me and donald  trump  my plans are focused
        

**Observations**

* Critical tone: ('trump tax cuts blew up', 'allowed Roe v. wade', 'dismissed them as headaches')
* Leadership criticism: ('trump being a threat to', 'openly admires dictators')
* Focus on specific actions ('pulled out of', 'handselected three members of', 'allowed Roe v. wade')
* Separation from Trump ('i'm not Donald Trump', 'between me and Donald trump')
* Media representation: ('trump campaign is a little', 'trump running tens of')

'Trump' mentions in Harris Podcasts

In [23]:
trump_harris_podcasts_kwic = make_kwic('trump', harris_podcasts_toks)
trump_harris_podcasts_random_sample = random.sample(trump_harris_podcasts_kwic, min(30, len(trump_harris_podcasts_kwic)))
print_kwic(trump_harris_podcasts_random_sample)

        virginia there is a  trump  abortion ban right and
  in people thinking donald  trump  was responsible for and
   does anyone think donald  trump  thinks that way thats
   the contrast here donald  trump  and his followers spent
    to our democracy donald  trump  would go in to
     who served with donald  trump  in the white house
    election i think donald  trump  is an unserious man
     were looking at donald  trump  basically somebody who has
    needed tests and donald  trump  during that time secretly
  are thinking about donald  trump  he was president and
   decision and then donald  trump  never being one to
             a state with a  trump  abortion ban you know
     the notion that donald  trump  could be president of
          fix it and donald  trump  told them hold on
military dont forget donald  trump  calling members of our
      example of who donald  trump  is that he secretly
       be very clear donald  trump  has been very clear
           which by the

**Observations**

* Use of Trump's full name
* Policy criticms ('died because of trump abortion bans'
* Character criticism ('unserious man', 'belittles', 'examples of who donald trump really is', 'secretly sent')
* Contrasts with others ('unlike any president before', 'trump abortion bans', 'trump who said of the')

### Emotional Language

‘Very’ mentions in Trump Interviews

In [24]:
very_trump_interviews_kwic = make_kwic('very', trump_interviews_toks)
very_trump_interviews_random_sample = random.sample(very_trump_interviews_kwic, min(30, len(very_trump_interviews_kwic)))
print_kwic(very_trump_interviews_random_sample)

      bob lighthizer did a  very  good job we had
         so far were doing  very  well i mean were
         was very rude sir  very  rude it was a
     people north korea is  very  nuclear they dont pay
           same time its a  very  dangerous thing because we
     who have been treated  very  unfairly because you have
          north korea is a  very  serious power they have
   country and people were  very  worried about — you
              me and i got  very  lucky — i really
   talking at another club  very  nice club and like
           know this he is  very  family oriented and he
           mean ive been a  very  successful businessman ive done
        yes maam and thats  very  clear now its very
  situation because it was  very  well documented in his
           know china is a  very  powerful very smart group
      tough you know whats  very  tough european union our
country left okay currency  very  important yes and if
             old and its a  very  different plan it

**Observations**

* Negative language: ('very sick', 'very hostile nasty', 'very disturbed', 'very rude'

‘Very’ mentions in Trump Podcasts

In [25]:
very_trump_podcasts_kwic = make_kwic('very', trump_podcasts_toks)
very_trump_podcasts_random_sample = random.sample(very_trump_podcasts_kwic, min(30, len(very_trump_podcasts_kwic)))
print_kwic(very_trump_podcasts_random_sample)

            we have a very  very  corrupt and hence the
         and this has been  very  interesting being with you
        got along with him  very  well which is a
    was you know certainly  very  strong in that sense
              and i had it  very  well sealed with the
               no it was a  very  nice phone call i
          ill get it going  very  quickly because not only
        use them theyll be  very  unsuccessful you know part
            with me it was  very  very strong very very
       hes very strong and  very  powerful and very interesting
              it but it is  very  disconcerting i said you
          found to be very  very  interesting how hes been
  very different story its  very  much i say an
to dominate theyre getting  very  big numbers i did
        know i mustve been  very  young it was a
          way you know its  very  interesting though so khabib
         great pilot and a  very  talented other pilots would
             ran and i was  very  close

**Observations**

* Negative Language: ('very liberal democrats', 'very bad', 'very corupt', 'very badly'

‘Very’ mentions in Harris Interviews

In [26]:
very_harris_interviews_kwic = make_kwic('very', harris_interviews_toks)
very_harris_interviews_random_sample = random.sample(very_harris_interviews_kwic, min(30, len(very_harris_interviews_kwic)))
print_kwic(very_harris_interviews_random_sample)

   election there are two  very  different choices one that
         years ago and im  very  clear that i will
       meet you thank you  very  much heres how i
          trump let me be  very  clear my presidency will
         the country i am  very  clear cost of groceries
           in 2020 i made  very  clear where i stand
      speaking and um its  very  humbling its very humbling
            all let me be  very  clear mine will not
immigration issue that is  very  significant to the american
           clear and i am  very  clear as is tim
   being presented with a  very  very serious decision about
          about it he was  very  clear that he was
       has done in making  very  important decisions on behalf
             to go and im  very  much grounded in the
    presented with a very  very  serious decision about what
          i believe it is  very  important that we take
          in favor of the  very  fundamental principle that a
   president well lets be  very  clear i

**Observations**

* Clarity: ([vert clear about where I stand', 'very clear my presidency will'

‘Very’ mentions in Harris Podcasts

In [27]:
very_harris_podcasts_kwic = make_kwic('very', harris_podcasts_toks)
very_harris_podcasts_random_sample = random.sample(very_harris_podcasts_kwic, min(30, len(very_harris_podcasts_kwic)))
print_kwic(very_harris_podcasts_random_sample)

              i do take it  very  seriously so there are
          the country in a  very  urban environment you know
          my husband i was  very  thoughtful and sensitive to
              and so i was  very  intentional about always making
          time oh ive been  very  clear i think that
    very happy marriage im  very  thankful to say like
strength democracy is also  very  fragile it will only
       five and just being  very  frank and candid with
          is very real and  very  much at stake in
       enemies he has been  very  clear that he would
          it listen i feel  very  strongly i need to
overturned people who felt  very  strong about that they
         of all i maintain  very  strongly america should never
          and lets also be  very  clear donald trump has
          you look in this  very  election legal black immigrants
            year it is its  very  like really real like
       forces that come in  very  different ways that are
        very touche

**Observations**

* Background/her life ('very modern family', 'very sick with cancer', 'my husband was very'

‘Good’ mentions in Trump Interviews

In [28]:
good_trump_interviews_kwic = make_kwic('good', trump_interviews_toks)
good_trump_interviews_random_sample = random.sample(good_trump_interviews_kwic, min(30, len(good_trump_interviews_kwic)))
print_kwic(good_trump_interviews_random_sample)

             i had a very  good  relationship with president xi
         me it didnt look  good  to me are you
             i did a very  good  job in covid nobody
             if i have 20  good  stories and 20 bad
     similar to ours very  good  product the phones and
         putin and a very  good  relationship with kim jongun
      hell he wants thats  good  way to negotiate let
        i was always very  good  at mathematics let me
    lighthizer did a very  good  job we had great
            know i have a  good  heart i have a
      and would have been  good  that thats a bad
       heard you got very  good  ratings on that interview
       it would have been  good  if it was nine
        he said all right  good  i said where do
            he said its a  good  product i said then
             — and a very  good  relationship with putin and
            of mine hes a  good  guy hes a smart
   good theyre doing very  good  i thought they were
something that didnt look  good  to m

‘Good’ mentions in Trump Podcasts

In [29]:
good_trump_podcasts_kwic = make_kwic('good', trump_podcasts_toks)
good_trump_podcasts_random_sample = random.sample(good_trump_podcasts_kwic, min(30, len(good_trump_podcasts_kwic)))
print_kwic(good_trump_podcasts_random_sample)

           fact this is a  good  way of me getting
            of them hes a  good  person they called me
          at a point very  good  despite the russia russia
           its for me its  good  fresh air it really
     he thinks this looks  good  because it looks so
        gonna have a very  good  election but i tell
   viral they were really  good  i think they were
             to do a very  good  job and maybe save
      good college a very  good  one and he was
            i never get a  good  story i only get
    my pollsters good guy  good  good guy and i
             i had a very  good  relationship thats a good
         with you whats a  good  one the ymca man
            gonna be in a  good  mood i hope so
          and i think hes  good  at what he does
             to be not so  good  but no the st
that slips forward sounds  good  nice book by the
     just pay my respects  good  luck have a good
             to be a very  good  shot my sons don
      a good student ver

‘Good’ mentions in Harris Interviews

In [30]:
good_harris_interviews_kwic = make_kwic('good', harris_interviews_toks)
good_harris_interviews_random_sample = random.sample(good_harris_interviews_kwic, min(30, len(good_harris_interviews_kwic)))
print_kwic(good_harris_interviews_random_sample)

  for the time its  good  to meet you thank
 you thank you its  good  to be with you
ill say that thats  good  work theres more to
   to do but thats  good  work no and i


‘Good’ mentions in Harris Podcasts

In [31]:
good_harris_podcasts_kwic = make_kwic('good', harris_podcasts_toks)
good_harris_podcasts_random_sample = random.sample(good_harris_podcasts_kwic, min(30, len(good_harris_podcasts_kwic)))
print_kwic(good_harris_podcasts_random_sample)

         yeah hes such a  good  person history is going
      but theres so much  good  in the world so
        we dont lack for  good  ideas but we do
      i believe you make  good  decisions i got to
     breaks my heart and  good  good teams because thats
       what made us feel  good  youre so right it
      hard work but it’s  good  work and it’s important
   seriously ive had the  good  fortune to mentor a
principles yeah but just  good  people right like good
    have to remember the  good  we have so many
 our warriors are always  good  whichever era but that
   people and things are  good  we are ambitious and
  good people right like  good  solid hardworking people and
      was not a terribly  good  businessman which is why
       is and whos doing  good  in the community what
  we should remember the  good  and i dont mean
     vacation and have a  good  time have nice christmas
have so many hardworking  good  people who i have
            you it is so  good  to be with y

‘Bad’ mentions in Trump Interviews

In [32]:
bad_trump_interviews_kwic = make_kwic('bad', trump_interviews_toks)
bad_trump_interviews_random_sample = random.sample(bad_trump_interviews_kwic, min(30, len(bad_trump_interviews_kwic)))
print_kwic(bad_trump_interviews_random_sample)

           done that are so  bad  he desperately wants immunity
            to me very very  bad  to me i mean
 difference between being a  bad  person and making an
           china all of the  bad  things and the evil
       allowed to put those  bad  votes and illegal votes
  standpoint they only have  bad  stories in other words
          good that thats a  bad  thing hes not saying
         all started with a  bad  energy policy by joe
        good stories and 20  bad  stories — and everyones
          well theyve had a  bad  period of time i
           wasnt so thats a  bad  thing and they were
             — that were so  bad  that youd have to
            only see the 20  bad  stories and i called
            and it was also  bad  steel it was what
           sense most of my  bad  stories but these are
         like biden whos in  bad  shape i wouldnt appoint
         deals that were so  bad  that i said who
             us on trade so  bad  — the european nations
           s

**Observations**

* Referring to opponent's bad choices

‘Bad’ mentions in Trump Podcasts

In [33]:
bad_trump_podcasts_kwic = make_kwic('bad', trump_podcasts_toks)
bad_trump_podcasts_random_sample = random.sample(bad_trump_podcasts_kwic, min(30, len(bad_trump_podcasts_kwic)))
print_kwic(bad_trump_podcasts_random_sample)

           taking all of their  bad  people really bad people
               hes got bad way  bad  way i looked at
           the people that are  bad  not only — i
      shes incompetent shes as  bad  as biden in a
                  bad he is so  bad  he is the worst
               look you have a  bad  debate his numbers went
                for so long in  bad  conditions you know in
              so bad they were  bad  all it was is
            no they know whats  bad  people either no its
          opposed to waltz how  bad  is he by the
          inflation its just a  bad  idea yeah they do
           we have some really  bad  people in our government
               it was a really  bad  month and a half
       neocons yeah neocons or  bad  people or disloyal people
            things they are so  bad  its like amateur night
              story i only get  bad  press now i will
         whole show is talking  bad  about trump yeah hes
             all they all have  bad  ratings 

**Observations**

* More intense emotion than in the interviews; ('so bad he is the worst', 'its just so bad its so sad')

‘Bad’ mentions in Harris Interviews

In [34]:
bad_harris_interviews_kwic = make_kwic('bad', harris_interviews_toks)
bad_harris_interviews_random_sample = random.sample(bad_harris_interviews_kwic, min(30, len(bad_harris_interviews_kwic)))
print_kwic(bad_harris_interviews_random_sample)

           no it was a  bad  debate people have bad
bad debate people have  bad  debates he is absolutely
  understand its not a  bad  thing as long as


‘Bad’ mentions in Harris Podcasts

In [35]:
bad_harris_podcasts_kwic = make_kwic('bad', harris_podcasts_toks)
bad_harris_podcasts_random_sample = random.sample(bad_harris_podcasts_kwic, min(30, len(bad_harris_podcasts_kwic)))
print_kwic(bad_harris_podcasts_random_sample)

    said im a really  bad  driver so thats how
   some child who is  bad  its not some child
 of class theyre the  bad  kid they still start
     to go after the  bad  actors where they exist
in an emergency some  bad  actors will jack up


### Other

'Problem' mentions in Trump Interviews

In [36]:
problem_trump_interviews_kwic = make_kwic('problem', trump_interviews_toks)
problem_trump_interviews_random_sample = random.sample(problem_trump_interviews_kwic, min(30, len(problem_trump_interviews_kwic)))
print_kwic(problem_trump_interviews_random_sample)


     one that has the  problem  i think theyre radical
      i would have no  problem  and you do have
    going to have any  problem  but youre going to
numbers and heres the  problem  we have some of
   else now heres the  problem  they say its unconstitutional


'Problem' mentions in Trump Podcasts

In [37]:
problem_trump_podcasts_kwic = make_kwic('problem', trump_podcasts_toks)
problem_trump_podcasts_random_sample = random.sample(problem_trump_podcasts_kwic, min(30, len(problem_trump_podcasts_kwic)))
print_kwic(problem_trump_podcasts_random_sample)

          now we have a  problem  because we have this
            — we had no  problem  with north korea we
       i dont have that  problem  but well first of
    get there thats the  problem  we have a number
          knew he had a  problem  and its a dont
       know there was a  problem  hes been hes an
       not gonna have a  problem  with russia russia would
          to me the big  problem  is the nuclear power
       not to have this  problem  and i couldnt have
     that doesnt have a  problem  shes a disaster shes
        and they have a  problem  and they say that
    united states its a  problem  and we have to
    and its obviously a  problem  and its a big
 said whats the biggest  problem  he said north korea
problem from within the  problem  on the outside if
  not and everythings a  problem  but its degrees to
           but i had no  problem  with it i did
      do with where the  problem  is you know she
      have to solve the  problem  from within the problem
   

'Problem' mentions in Harris Interviews

In [38]:
problem_harris_interviews_kwic = make_kwic('problem', harris_interviews_toks)
problem_harris_interviews_random_sample = random.sample(problem_harris_interviews_kwic, min(30, len(problem_harris_interviews_kwic)))
print_kwic(problem_harris_interviews_random_sample)

   problem not promoting a  problem  ok and the numbers
       more about fixing a  problem  even if it is
               to run on a  problem  instead of fixing a
               to run on a  problem  instead of fixing a
   forward i recognize the  problem  dana i think the
        are about fixing a  problem  not promoting a problem
       instead of fixing a  problem  and in this election
       instead of fixing a  problem  and again in this
   obstacle to fixing this  problem  my proposal includes strengthening
          been on fixing a  problem  and from day one
       instead of fixing a  problem  so he told his
               to run on a  problem  instead of fixing a
forward its a longstanding  problem  and solutions are at
       to actually fix the  problem  im glad youre pointing


'Problem' mentions in Harris Podcasts

In [39]:
problem_harris_podcasts_kwic = make_kwic('problem', harris_podcasts_toks)
problem_harris_podcasts_random_sample = random.sample(problem_harris_podcasts_kwic, min(30, len(problem_harris_podcasts_kwic)))
print_kwic(problem_harris_podcasts_random_sample)

       a solution to a  problem  which is that we
        it would fix a  problem  and told his buddies
      came home with a  problem  im telling you every
   instead of fixing a  problem  and hes running his
     heres part of the  problem  with social security is
   instead of fixing a  problem  and is putting out
      instead of fix a  problem  and weve got to
      came home with a  problem  the first thing my
           to run on a  problem  instead of fix a
           to run on a  problem  instead of fixing a
affordable part of the  problem  there is we have
           to run on a  problem  instead of fixing a


'Example' mentions in Trump Interviews

In [40]:
example_trump_interviews_kwic = make_kwic('example', trump_interviews_toks)
example_trump_interviews_random_sample = random.sample(example_trump_interviews_kwic, min(30, len(example_trump_interviews_kwic)))
print_kwic(example_trump_interviews_random_sample)

     give me a quick  example  and then i will
   could give you an  example  of this if you
    in chicago as an  example  a few weeks ago
ill give you another  example  we have a man


'Example' mentions in Trump Podcasts

In [41]:
example_trump_podcasts_kwic = make_kwic('example', trump_podcasts_toks)
example_trump_podcasts_random_sample = random.sample(example_trump_podcasts_kwic, min(30, len(example_trump_podcasts_kwic)))
print_kwic(example_trump_podcasts_random_sample)

       ill give you an  example  i came out when
 pipp now the ultimate  example  of that was didnt
       ill give you an  example  were gonna lose our
      mark levin is an  example  they couldnt believe it
       this list as an  example  so its such a
        ill take as an  example  any one of you
         of them as an  example  venezuela their crime is
       japan was as an  example  theyll send us millions
you take venezuela for  example  venezuela should be incredibly
     about trump as an  example  you interview a lot
          they — as an  example  the way they pulled
interesting like as an  example  tiger or jack nicklaus
      hoax thats a big  example  but thats only one


'Example' mentions in Harris Interviews

In [42]:
example_harris_interviews_kwic = make_kwic('example', harris_interviews_toks)
example_harris_interviews_random_sample = random.sample(example_harris_interviews_kwic, min(30, len(example_harris_interviews_kwic)))
print_kwic(example_harris_interviews_random_sample)

   of leadership i for  example  am someone who has
price of groceries for  example  dealing with an issue
the private sector for  example  home builders to create
invest in families for  example  extending the child tax
      i make about for  example  my plan for increasing
   gas emissions as an  example  that value has not
these are examples the  example  is what im going


'Example' mentions in Harris Podcasts

In [43]:
example_harris_podcasts_kwic = make_kwic('example', harris_podcasts_toks)
example_harris_podcasts_random_sample = random.sample(example_harris_podcasts_kwic, min(30, len(example_harris_podcasts_kwic)))
print_kwic(example_harris_podcasts_random_sample)

       be addressed so for  example  as only the second
     the most recent stark  example  of who donald trump
   supportive about so for  example  here in atlanta what
         of the things for  example  on domestic violence that
        that have gone for  example  to public servants like
     of home ownership for  example  black people are 40
          every day im for  example  one of my passions
     well its just another  example  of him trying to
   debt and thankfully for  example  we have billions of
        do to increase for  example  the high risk that
          when we like for  example  i helped write the
  deal with addressing for  example  my work around what
very intentional about for  example  knowing that the kids
          are a couple for  example  and one of the
      ill give you another  example  of why donald trump
        of my policies for  example  is to help medicare
        growth my plan for  example  okay so some of
         going to talk for  exampl

**Observations**

* In podcasts, Harris gives more examples of Donald Trump's problems that she does in interviews